In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

from torch.utils.data import Dataset, DataLoader
import time
from torch.cuda.amp import autocast, GradScaler

In [2]:
cfg = {
    "vocab_size" : 50257,
    "context_length": 1024,
    "emb_dim" : 768,
    "n_layers" : 12,
    "n_heads" : 12,
    "drop_out" : 0.1,
    "qkv_bias": True
}

## Multi Head Attention

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=True):
        super().__init__()
        
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, float("-inf"))
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

## Layer normalization, GELU activation and feed forward network

In [4]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))


class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]), ## Expansion
            GELU(), ## Activation
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]), ## Contraction
        )

    def forward(self, x):
        return self.layers(x)

## Transformer block 

In [5]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"], 
            dropout=cfg["drop_out"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_out"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        # 2*4*768
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x
        # 2*4*768

## GPTModel

In [6]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_out"])
        
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        
        self.final_norm = LayerNorm(cfg["emb_dim"])
        
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)
        # weight tying scheme
        self.out_head.weight = self.tok_emb.weight

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

## device agnostic code

In [7]:
device = "cuda" if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Creating an instance of GPTModel class

In [8]:
gpt = GPTModel(cfg)

In [9]:
gpt_size = sum(p.numel() for p in gpt.parameters())
print(f"GPT2 total parameters: {gpt_size/1000000:.4f}M")

GPT2 total parameters: 124.4398M


In [10]:
gpt.load_state_dict(torch.load('../model1.pth'))

<All keys matched successfully>

In [11]:
def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=50256):

    # print("using device: ", device)
    model.to(device)

    # For-loop is the same as before: Get logits, and only focus on last time step
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]

        # New: Filter logits with top_k sampling
        if top_k is not None:
            # Keep only top_k values
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(float("-inf")).to(logits.device), logits)

        # New: Apply temperature scaling
        if temperature > 0.0:
            logits = logits / temperature

            # Apply softmax to get probabilities
            probs = torch.softmax(logits, dim=-1)  # (batch_size, context_len)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (batch_size, 1)

        # Otherwise same as before: get idx of the vocab entry with the highest logits value
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch_size, 1)

        if idx_next.item() == eos_id:
            break

        # Same as before: append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch_size, num_tokens+1)

    return idx

In [19]:
input = "What basketball"
encoded = torch.tensor(tokenizer.encode(input), device=device).unsqueeze(0)
new_tok = generate(gpt, encoded, max_new_tokens=40, context_size=cfg['context_length'], temperature=0.7, top_k=50).squeeze()
new_tok = new_tok.tolist()

print(tokenizer.decode(new_tok))

What basketball is? Basketball is a sport that is played on a court with two hoops, each 42-minute long range on a court. In the sport, it is a sport played on a pitch with a


In [93]:
## I trained the pre-trained gpt2 on some general dataset which consist of general knowledge
## Now the model is performing good, but it takes few tries to get correct defination

In [99]:
## this class helps to prepare the dataset in numerical form, which returns x and y (x inputs, y targets which is shifted by one position)
## this is also called as preprocessing the data, where according to the data we can modify and feed it to our model
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={'<|endoftext|>'})
        for i in range(0, len(token_ids)- max_length, stride):
            input_chunk = token_ids[i: i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk, dtype=torch.long))
            self.target_ids.append(torch.tensor(target_chunk, dtype=torch.long))

            
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [100]:
def create_dataloader_v1(txt, batch_size=8, max_length=256, stride=128, shuffle=False, drop_last=True, num_workers=0):

    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, 
                            batch_size=batch_size,  
                            shuffle=shuffle, 
                            drop_last=drop_last, 
                            num_workers=num_workers)
    return dataloader
    

In [101]:
# reading a file
import re
with open('../LLM_from_scratch/TrainingData/dollyData_cleaned.txt', encoding='utf-8') as f:
    raw_text = f.read()
len(raw_text)

11955778

In [102]:
# splitting the training and validation part
split = int(len(raw_text) * 0.95)
train_data = raw_text[:split]
test_data = raw_text[split:]

In [103]:
train_loader = create_dataloader_v1(train_data, batch_size=6, max_length=256, stride=128, shuffle=False, drop_last=True, num_workers=0)
val_loader = create_dataloader_v1(test_data, batch_size=6, max_length=256, stride=128, shuffle=False, drop_last=True, num_workers=0)
len(train_data), len(test_data)

(11357989, 597789)

In [104]:
# couting how many pairs of input and target are there in total
count = 0
for input, target in train_loader:
    count += 1
    # print(input.shape,target.shape)
    # print(f"input: {input} \ntarget: {target}")
print(count)

3179


In [105]:
# creating optimizer and loss fucntion

import time
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(gpt.parameters(), lr=3e-4, weight_decay=1e-2)

In [107]:
# training starts here
epochs = 10
device = "cuda" if torch.cuda.is_available() else "cpu"

gpt.to(device)
scaler = torch.amp.GradScaler('cuda')

start_time = time.perf_counter()

for epoch in range(epochs):
    gpt.train()
    total_train_loss = 0.0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()

        # Forward + loss in mixed precision
        with torch.amp.autocast('cuda'):
            logits = gpt(x)
            logits = logits.reshape(-1, logits.size(-1))
            y = y.reshape(-1)
            loss = loss_fn(logits, y)

        # Backward with scaled loss
        scaler.scale(loss).backward()

        # Clip gradients (unscale first)
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(gpt.parameters(), max_norm=1.0)

        # Optimizer step with scaler
        scaler.step(optimizer)
        scaler.update()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    # Validation
    gpt.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            with torch.amp.autocast('cuda'):
                logits = gpt(x)
                logits = logits.reshape(-1, logits.size(-1))
                y = y.reshape(-1)
                val_loss = loss_fn(logits, y)
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    
    end_time = time.perf_counter()
    total_time = (end_time - start_time)

    print(f"Epoch {epoch+1} | "
          f"Train Loss: {avg_train_loss:.4f} | "
          f"Val Loss: {avg_val_loss:.4f} | "
          f"time: {total_time:.2f} sec")

###### Epoch 1 | Train Loss: 2.5036 | Val Loss: 3.4717 | time: 11739.86 sec
###### Epoch 2 | Train Loss: 2.0603 | Val Loss: 3.5833 | time: 23478.83 sec
###### Epoch 3 | Train Loss: 1.6987 | Val Loss: 3.7992 | time: 35220.40 sec
###### Epoch 4 | Train Loss: 1.4174 | Val Loss: 4.0462 | time: 46968.51 sec
###### Epoch 5 | Train Loss: 1.1946 | Val Loss: 4.4120 | time: 58711.07 sec
###### Epoch 6 | Train Loss: 1.0185 | Val Loss: 4.6628 | time: 70452.31 sec
###### Epoch 7 | Train Loss: 0.8773 | Val Loss: 4.9774 | time: 82193.95 sec
###### Epoch 8 | Train Loss: 0.7647 | Val Loss: 5.2615 | time: 93933.10 sec
###### Epoch 9 | Train Loss: 0.6744 | Val Loss: 5.5343 | time: 105671.69 sec
###### Epoch 10 | Train Loss: 0.6025 | Val Loss: 5.7923 | time: 117409.76 sec

## In total it took 117409.76 sec roughly 32.5 hours to train (dataset size is 11.42mb)
##### Probably i could increase the batch size to reduce the time, but due to limited vram i stick with batch size of 6

In [120]:
# testing the model
input = "what is java"
encoded = torch.tensor(tokenizer.encode(input), device=device).unsqueeze(0)
new_tok = generate(gpt, encoded, max_new_tokens=40, context_size=cfg['context_length'], temperature=0.7, top_k=50).squeeze()
new_tok = new_tok.tolist()
tokenizer.decode(new_tok)

'what is java a programming language that is very high-level and very easy to learn.  It was invented as a high-level programming language that was used to encourage fast and easy coding skills development. '

In [121]:
## saving the model's parameters 
torch.save(gpt.state_dict(), "model2.pth")